# Python-1, Лекция 11

Лектор: Хайбулин Даниэль

Подготовил материал: Лущ Иван

Итак, сегодня мы поговорим про простанства имен и декораторы

### [Namespaces](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)

Предыдущее занятие о функциях и классах дало интуитивное понимание локальных и глобальных переменных. Сегодня формализуем понятие "пространство имен" и механизмы разрешения имен в Python.

Под **пространством имен** (`namespace`) будем понимать отображение имя -> объект.

**Область видимости** (`scope`) — это текстовая область программы на Python, в пределах которой пространство имён доступно напрямую. "Доступно напрямую" означает, что обращение к имени пытается найти это имя в данном пространстве имён.


In [ ]:
# global namespace
# scope
x = 'модуль'

def f():
    x = 'локальная'
    # достали x из local namespace
    print('в f:', x)
    # scope здесь

# f создает новый local namespace при вызове
f()

Блоки `if`/`for`/`while`/`with` новых областей видимости не создают; функция (и `lambda`) при вызове — создает; `comprehension`/генераторные выражения в `Python 3` создают свой маленький локальный `scope` для переменной цикла.

Что будет?

In [ ]:
funcs = [lambda: i for i in range(3)]
print([f() for f in funcs])

lambda захватывает не значение i, а саму переменную i (ячейку) из enclosing‑scope comprehension.

<div style="
    background-color: #FFBA00;
    padding: 15px;
    border-left: 5px solid #ffcc00;
    text-align: center;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 15px;
">
    <span style="color: white; font-weight: bold;">
        Лучше не использовать lambda, обсуждали в конце лекции "Ссылки. Изменяемость."
    </span>
</div>

Python ищет имя по порядку `LEGB`: сначала в текущей локальной области (`L`, `Local`), потом во внешних функциях (`E`, `Enclosing`), потом в модуле (`G`, `Global`), потом во встроенных именах (`B`, `Builtins`).

![](legb.png)


- `Global`: содержит `funcs`
- `Enclosing` (охватывающее для `lambda`): каждое `list‑comprehension` в Python исполняется в своём локальном `scope`; переменная `i` — локальная именно для этого скрытого "функционального" `scope comprehension`.
- `Local` (у `lambda`): появляется только при вызове `lambda`; внутри тела `lambda` нет собственного `i`, поэтому поиск идёт во внешнее (`enclosing`) пространство.
- `Builtins`: здесь находятся `print` и `range`

Фикс через аргумент по умолчанию:

In [ ]:
funcs = [lambda i=i: i for i in range(3)]
print([f() for f in funcs])

Мы начали с необычного примера, чтобы интуитивно увидеть: имя ищется по цепочке областей видимости, а замыкания захватывают не значение, а ячейку имени. Теперь разберём формально каждое звено этой цепочки.

`builtins namespace`:

- Набор "встроенных" имён: `print`, `range`, `len`, `int`, `str`, `Exception`, `True`/`False`/`None`, ...
- Cоздаётся при инициализации интерпретатора; один на интерпретатор.
- Как используется: последнее звено в `LEGB`; имена доступны без импорта.

In [ ]:
print(dir(__builtins__))

In [ ]:
import builtins
builtins.print(dir(builtins))

`global namespace`:
- Что это: словарь `globals()` конкретного модуля (включая `__name__`, `__builtins__`, ваши функции/классы/переменные).
- Когда создаётся: при выполнении модуля (импорт или запуск).
- Время жизни: пока живёт модуль (обычно до завершения интерпретатора).


In [ ]:
globals()

`enclosing namespace`:
- Что это: пространство имён внешней функции (и вложенных уровней), доступное для внутренних функций.
- Когда создаётся: при вызове внешней функции (или при выполнении `comprehension`).
- Время жизни: пока есть активные ссылки.
